<a href="https://colab.research.google.com/github/KatherinneOlaya/Prompt-Engineering/blob/main/RAG_Chunk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [45]:
import shutil
shutil.copy("/content/drive/MyDrive/Ai generative/attention-is-all-you-need-Paper.pdf", "doc.pdf")
print("PDF cargado!")

PDF cargado!


In [8]:
%%capture
!pip install pypdf

In [46]:
 # ============= Extraer PDF =======================#

from pypdf import PdfReader
read = PdfReader("doc.pdf")
number_of_pages = len(read.pages)

text =""
for page in read.pages:
  text += page.extract_text()



In [47]:
# ============= Chunks ===========================#

def chunks(text, max_len = 300):
  chunks_list = []
  actual = []
  words = text.split()

  for word in words:
    if len(" ".join(actual)) + 1 + len(word) <= max_len:
      actual.append(word)
    else:
      chunks_list.append(" ".join(actual))
      actual = [word]

  if actual:
    chunks_list.append(" ".join(actual))

  return chunks_list

chunk = chunks(text)

In [25]:
%%capture
!pip install groq


In [48]:
from groq import Groq
client = Groq(api_key="")


In [ ]:
#================= PROMPT =============================

In [49]:
def buscar(query):
    relevantes = [c for c in chunk if query.lower() in c.lower()]
    return relevantes[:5]  # Top 5

In [50]:
def rag(question):
  contextos = buscar(question)

  prompt = f"""
    CONTEXTO:
    {' '.join(contextos)}

    PREGUNTA:
    {question}

    REGLAS:
    - Responde únicamente usando el contexto.
    - No inventes información.
    """
  resp = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role":"user", "content": prompt}]
)

  return resp.choices[0].message.content

In [54]:
rag("Cual es la explicacion de Attention is all you need")

'"Attention is All You Need" es un artículo de investigación publicado en 2017 por Vaswani, Shazeer, Parmar, Uszkoreit, Jones, Gomez, Kaiser y Polosukhin. En el artículo, los autores presentan una nueva arquitectura de modelo de lenguaje llamada Transformers (Transformadores).\n\nLa idea central del artículo es que tradicionalmente, los modelos de lenguaje utilizaban secuencialidad y se basaban en redes neuronales recurrentes (RNN) o redes neuronales convolucionales (CNN) para procesar secuencias de tokens. Sin embargo, estas arquitecturas no pueden manejar secuencias de longitud extensa de manera eficiente.\n\nLos autores proponen una nueva forma de atención llamada "Attention is All You Need", que utiliza una función de atención que permite al modelo considerar todas las posibles combinaciones de tokens en una secuencia al mismo tiempo. Esto permite al modelo capturar patrones y relaciones globales en la secuencia sin necesidad de recorrer la secuencia de manera secuencial.\n\nLa arq